In [155]:
#пункт 1

using LinearAlgebra

function forwardsub(L, b) 
    n = size(L, 1)
    x = zeros(n)
    
    if (size(L,1)!= size(L,2)) || (size(L,1)!=size(b, 1))
        println("неправильная размерность")
        return x
    end
    
    if ( isapprox(prod(diag(L)), 0.0) )
        println("Вырожденная матрица")
        return x
        
    end
    
    sum = 0.0
    
    for i in 1:n-1
        for j in i+1:n
            sum+=L[i,j]^2
        end
    end
    
    if ( !isapprox(0.0, sum) )
        println("Не нижняя диагональная")
        return x
    end
    
    x[1] = b[1]/L[1,1]
    if (n>1)
        for i in 2:n
            s = 0.0
            for j in 1:i-1
                s+=L[i,j]*x[j]
            x[i] = (b[i] - s)/L[i,i]
            end
        end
    end
    
    return x

end





function backwardsub(U, b)
    n = size(U, 1)
    x = zeros(n)
    
    if (size(U,1)!= size(U,2)) || (size(U,1)!=size(b, 1))
        println("неправильная размерность")
        return x
    end
    
    if ( isapprox(prod(diag(U)), 0.0) )
        println("Вырожденная матрица")
        return x
        
    end
    
    sum = 0.0
    
    for i in 1:n-1
        for j in i+1:n
            sum+=U[j,i]^2
        end
    end
    
    println(sum)
    if ( !isapprox(0.0, sum) )
        println("Не верхняя диагональная")
        return x
    end
    
    x[n] = b[n]/U[n,n]
    if (n>1)
        for i in 1:n-1
            s = 0.0
            
            for j in 1:i
                
                s+=U[n-i,n-j+1]*x[n-j+1]
                
            
            end
            x[n-i] = (b[n-i] - s)/U[n-i,n-i]
        end
    end
    
    return x
end




backwardsub (generic function with 1 method)

In [157]:
# test forward(L,b)
a = [
    1 0 0; 
    4 5 0; 
    1 4 1]

b = [1, 2, 3]

x = forwardsub(a, b)

println(x)

c =b - a*x

println(c)


[1.0, -0.4, 3.6]
[0.0, 0.0, 0.0]


In [158]:
#test backward(U,b)
a = [
    1 9 3; 
    0 1 1; 
    0 0 1]

b = [1, 2, 3]

x = backwardsub(a, b)

println(x)

c =b - a*x

println(c)




0.0
[1.0, -1.0, 3.0]
[0.0, 0.0, 0.0]


In [24]:
#Пункт 2
using LinearAlgebra

function tridiagsolve(a::Vector, b::Vector, c::Vector, f::Vector)
    
    n = size(b,1)
    x = zeros(n)
    
    b_new = zeros(n)
    f_new = zeros(n)
    
    b_new[1] = b[1]
    f_new[1] = f[1]
    
    for i in 2:n
        
        b_new[i] = b[i] - a[i]*c[i-1]     / b_new[i-1]
        f_new[i] = f[i] - a[i]*f_new[i-1] / b_new[i-1]
  
    end
    
    x[n] = f_new[n]/b_new[n]
    
    for i in 1:n-1
        
        x[n-i] = (f_new[n-i] - c[n-i]*x[n-i+1])/b_new[n-i]
    
    end
    
    return x

end


function make_matrix(a,b,c)
   
    n = size(b,1)
    A = zeros(n,n)
    
    for i in 1:n
        A[i,i] = b[i]
    end
    
    for i in 1:n-1
       A[i,i+1] = c[i] 
    end
    
    for i in 2:n
        A[i,i-1] = a[i]
    end
    
return A
    
end


function tridiagsolve(A::Tridiagonal, f)
    
    n = size(A,1)
    a = zeros(n)
    b = zeros(n)
    c = zeros(n)
    
    for i in 1:n
        b[i] = A[i,i]
    end
    
    for i in 1:n-1
       c[i] = A[i,i+1] 
    end
    
    for i in 2:n
        a[i] = A[i,i-1]
    end
    
    return tridiagsolve(a, b, c, f)
end

tridiagsolve (generic function with 2 methods)

In [23]:
#test2 for 1st method

a = [0, 2, 1, 1, 1, 1]
b = [1, 5, 1, 6, 1, 1]
c = [1, 1, 5, 1, 51, 0]
f = [1, 2, 3, 4, 5, 6]

x = tridiagsolve(a, b, c, f)

println(x)

A = make_matrix(a,b,c)
for i in 1:size(A,1)
    println(A[i,:])
end
println(A*x - f)

[-8.513513513513518, 9.513513513513518, -28.54054054054055, 4.405405405405407, 6.108108108108108, -0.10810810810810813]
[1.0, 1.0, 0.0, 0.0, 0.0, 0.0]
[2.0, 5.0, 1.0, 0.0, 0.0, 0.0]
[0.0, 1.0, 1.0, 5.0, 0.0, 0.0]
[0.0, 0.0, 1.0, 6.0, 1.0, 0.0]
[0.0, 0.0, 0.0, 1.0, 1.0, 51.0]
[0.0, 0.0, 0.0, 0.0, 1.0, 1.0]
[0.0, 0.0, 3.552713678800501e-15, -8.881784197001252e-16, 8.881784197001252e-16, 0.0]


In [26]:
#test2 for 2nd method
dl = [1, 2, 3]
d  = [5, 6, 7, 8]
du = [9, 8, 7]
A = Tridiagonal(dl, d, du)
f = [1, 2, 3, 4]

x = tridiagsolve(A, f)
println(x)

println(A*x - f)



[-8.799999999999999, 4.999999999999999, -2.3999999999999995, 1.4]
[0.0, 0.0, 0.0, 8.881784197001252e-16]


In [34]:
#Пункт 3  TESTS

function solution_a()
    A = [
        8 9 4 1;
        0 4 1 0;
        0 0 -1 6;
        0 0 0 11 ]
    
    b = [9, 3, -1, 2]
    x = A \ b
    return x, b - A * x
end

println("a: solution $(solution_a()[1]); residual $(solution_a()[2])")


function solution_b()
    A = [
        -2 1 0 0 0;
        1 -2 1 0 0;
        0 1 -2 1 0;
        0 0 1 -2 1;
        0 0 0 1 -2]
    
    b = [1, 1, 1, 1, 1]
    x = A \ b
    return x, b - A * x
end

println("b: solution $(solution_b()[1]); residual $(solution_b()[2])")


function solution_c()
    A = [
        1 8 -3 9;
        0 4 10 -2;
        8 2 -5 1;
        3 1 6 12]
    
    b = [3, 6, 1, 4]
    x = A \ b
    return x, b - A * x
end

println("c: solution $(solution_c()[1]); residual $(solution_c()[2])")




a: solution [-0.19886363636363635, 0.2272727272727273, 2.090909090909091, 0.18181818181818182]; residual [0.0, 0.0, 0.0, 0.0]
b: solution [-2.5, -3.9999999999999996, -4.499999999999999, -4.0, -2.5]; residual [-4.440892098500626e-16, 0.0, 1.7763568394002505e-15, -8.881784197001252e-16, 0.0]
c: solution [0.2629310344827586, 0.4741379310344827, 0.41451149425287354, 0.02083333333333337]; residual [0.0, 8.881784197001252e-16, 1.1102230246251565e-16, 0.0]
